In [1]:
# !pip install wordcloud

In [1]:
%run -i helpers.py
%matplotlib inline

[nltk_data] Downloading package punkt to /Users/kamal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
frames = []
for files in os.listdir('./tickers'):
    if re.search('\.csv',files):
        df = pd.read_csv('./tickers'+ '/' +files, index_col=False)
        frames.append(df)

final_df = pd.concat(frames)

In [3]:
id_cols = ['ticker', 'q', 'Name', 'Country', 'Sector','Industry']

In [4]:
cols_expect = ['Environmental','Social','Governance']
esg_labels = ['Environmental','Social','Governance']

In [5]:
tic_meta_df = pd.read_csv('./data/tic_names.csv')[['Symbol', 'Name',
       'Country', 'Sector', 'Industry']].rename(columns={'Symbol':'ticker'})
final_df = final_df.merge(tic_meta_df, how = 'left', on = 'ticker')
final_df.Name = final_df.Name.apply(lambda x: str(x).replace('Common Stock','').replace('Common Shares',''))

In [6]:
ai_df = pd.DataFrame()

## Contradiction Analysis

In [7]:
# find candidates
def check_diff_esg_cols(df):
    retdf = []
    for c in cols_expect:
        diff = np.nansum((df[c+'_QnA']-df[c+'_prep_remarks']))*100/df.shape[0]
        retdf.append([df.ticker.iloc[0],df.Name.iloc[0],df.Sector.iloc[0],df.Industry.iloc[0],diff,c])
    df = pd.DataFrame(retdf, columns=['ticker', 'Name','Sector','Industry','pdiff','label'])
    return(df)

# pdiffdf = final_df.groupby(['ticker','dt']).apply(check_diff_esg_cols).droplevel(0)   # dt gives more False positives 
pdiffdf = final_df.groupby(['ticker']).apply(check_diff_esg_cols).droplevel(0)
gw_candidates = pdiffdf[(abs(pdiffdf.pdiff)>50)]
# gw_candidates.reset_index(inplace=True)  # only reqd if you apply on dt 
if('level_1' in gw_candidates.columns):
    gw_candidates.drop(columns='level_1',inplace=True)

In [8]:
# gw_candidates.groupby('ticker')['dt'].nunique()
# there are cases where companies would consistently have conflicting claims 

In [9]:
## Find conflicting claims
def get_text_example_section_sing_tr(tr, neg=True, label = 'Environmental', section='QnA', sent_thresh = 0.7):
    # neg=param_dict['neg'] 
    # label = param_dict['label'] 
    # section=param_dict['section'] 
    # sent_thresh = param_dict['sent_thresh']
    
    if(neg==True):
        label_scores_df = get_esg_label_transcript(tr)
        label_scores_df['sentiment'] = label_scores_df.sent.apply(lambda x: create_sentiment_output(sentiment_pipeline(x)))
        label_scores_df.sentiment = label_scores_df.sentiment.apply(lambda x:np.round(x,4))
        clean_scores = label_scores_df[((label_scores_df.label_score>sent_thresh) & (label_scores_df.sentiment!=0))]
        reslist = clean_scores[((clean_scores.sentiment<0) &(clean_scores.esg_label==label))].sent.tolist()
        return(list(set(reslist)))
    else:
        label_scores_df = get_esg_label_transcript(tr)
        label_scores_df['sentiment'] = label_scores_df.sent.apply(lambda x: create_sentiment_output(sentiment_pipeline(x)))
        label_scores_df.sentiment = label_scores_df.sentiment.apply(lambda x:np.round(x,4))
        clean_scores = label_scores_df[((label_scores_df.label_score>sent_thresh) & (label_scores_df.sentiment!=0))]
        reslist = clean_scores[((clean_scores.sentiment>0) &(clean_scores.esg_label==label))].sent.tolist()
        return(list(set(reslist)))
    
def get_text_example_for_tic_section(df, neg=True, label = 'Environmental', section='QnA', sent_thresh = 0.7):
    # param_dict = dict()
    # param_dict['neg'] = neg
    # param_dict['label']= label 
    # param_dict['section'] = section
    # param_dict['sent_thresh'] = sent_thresh

    res = []
    tr_sents = df.QnA.apply(get_text_example_section_sing_tr, args = (neg,label,section,sent_thresh))
    for x in tr_sents:
        for l in x:
            res.append(l) 
    return(res)

In [10]:
# @interact
# def func(company_tic = widgets.Dropdown(options=gw_candidates.ticker.tolist(),value='ACA'), 
#          label = widgets.Dropdown(options=esg_labels,value='Environmental')):
#     return

In [11]:
@interact
def func(company_tic = widgets.Dropdown(options=gw_candidates.ticker.tolist(),value='ACA'), 
         label = widgets.Dropdown(options=esg_labels,value='Environmental')):
    print('\n'*2)
    print(f'Some {label} Greenwashing candidates')
    display(gw_candidates[gw_candidates.label==label].sort_values('pdiff',ascending=False,key=abs).head(50))
    chosen_candidate_df = final_df[((final_df.ticker==company_tic) &(final_df[label+'_QnA']<0))]

    print('\n')
    print(f'\n {label} scores for {company_tic}:')
    fig, ax = plt.subplots(figsize=(8,5))
    tdf = final_df[final_df.ticker==company_tic]
    for lbl in esg_labels:
        for s in ['_QnA', '_prep_remarks']:
            if(lbl==label):
                ax.plot('q', lbl+s, data = tdf)
    plt.ylabel(f'{label} scores for {company_tic}')
    ax.legend()
    plt.xticks(rotation=45, ha='right')
    plt.show()
    
    print('\n')
    display(widgets.Text(value='Risks/ Conflicting statements from QnA sections',placeholder='Risks'))
    display(widgets.Text(value='Optimism in prepared remarks',placeholder='Risks'))
    sent_eg = get_text_example_for_tic_section(chosen_candidate_df, True, label, section='QnA', sent_thresh = 0.7)

    tab_contents = ['Risks', 'Prep_remarks']
    children = [widgets.HTML(value=f"<b>{name}",description=name) for name in tab_contents]
    tab = widgets.Tab()
    tab.children = children
    tab.set_title(0, 'Risks')
    tab.set_title(1, 'Prepared remarks')
    
    wordcloud_data_statements = get_text_example_for_tic_section(chosen_candidate_df, False, label, section='prep_remarks', 
                                                                 sent_thresh = 0.7)
    
    children[0].value = '<br>'.join(sent_eg)
    children[1].value = '<br>'.join(wordcloud_data_statements)

    display(tab)

    print('\n\n Wordcloud on the prepared remarks\n')

    # print('\n'*2)
    if(len(wordcloud_data_statements)>0):
        get_wordcloud(wordcloud_data_statements)
        plt.show()
    
    print('\n')
    display(widgets.Text(value='Responses from AI LLM',placeholder='AI'))
    print('\n')
    ddf = ai_df[ai_df.ticker==company_tic]
    if(ddf.shape[0]>0):
        display(widgets.HTML(value=f"<b>{ddf.ai_response.iloc[0]}",description='AI Response'))
    else:
        print('No cached response from AI')
    
    print('\n'*2)
    print(f'Total # of companies: {final_df.ticker.nunique()} and we found conflicitng statements in {gw_candidates.ticker.nunique()}')
    sector_ct = gw_candidates[gw_candidates.label==label].groupby('Sector')['ticker'].nunique().reset_index()\
                                                            .rename(columns={'ticker':'companies_detected'})
    secdfall = final_df.groupby('Sector')['ticker'].nunique().reset_index().rename(columns={'ticker':'total_num_companies'})
    secdfall = secdfall.merge(sector_ct, how='left',on='Sector' )
    # secdfall.dropna(inplace=True)

    secdfall.plot(x='Sector', y=['total_num_companies','companies_detected'], kind='bar', figsize=(10,5)).legend( bbox_to_anchor =(1 ,1))
    
    
    return

interactive(children=(Dropdown(description='company_tic', index=1, options=('ABOS', 'ACA', 'ADAP', 'ADM', 'ADN…

In [12]:
# sector_scores = final_df.groupby(['Sector','q','dt']).agg({'Environmental_prep_remarks':np.nanmedian, 'Social_prep_remarks':np.nanmedian, 
#                                            'Governance_prep_remarks':np.nanmedian}) .reset_index()

In [13]:
# sector_scores[sector_scores.Sector=='Energy']

In [14]:
# ## Testing 
# label = 'Environmental'
# choose_company = 'ACA'
# chosen_candidate_df = final_df[final_df.ticker==choose_company]
# chosen_candidate_df.plot()

# print('\n'*2)
# print(f'ESG scores for {choose_company}:')
# fig, ax = plt.subplots(figsize=(8,5))
# tdf = final_df[final_df.ticker==choose_company]
# for lbl in esg_labels:
#     for s in ['_QnA', '_prep_remarks']:
#         if(lbl==label):
#             ax.plot('q', lbl+s, data = tdf)
# plt.xticks(rotation=45, ha='right')

# ax.legend()
# plt.show()


In [15]:
# get_sentiment_eg(final_df[((final_df.ticker=='AAPL') &(final_df['Environmental'+'_QnA']<0))],True)

In [ ]:
    # print('\n ### Risks/ Conflicting statements from QnA sections ###')
    # display(pd.DataFrame(sent_eg, columns=['conflicting_statements_qna']))
    # print(sent_eg)
